In [2]:
from urllib.parse import quote 
from timeit import default_timer as timer
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

t1_start = timer()
# Initialize embedding
embedding = HuggingFaceEmbeddings(cache_folder="./modal")
# Initialize chroma persistentClient
db = Chroma(collection_name="sql-wiki", embedding_function=embedding, persist_directory="./chroma")
t1_end = timer()

# Similarity search
# qt_start = timer()
# query = "How do I fix backup error?"
# # query = "How to create new debit note?"
# # query = "How to import from autocount?"
# # result = db.similarity_search(query)
# # Return 5 result with score more than 0.8
# # result = db.similarity_search_with_relevance_scores(query=query, k=5, kwargs={"score_threshold": 0.8})
# result = db.similarity_search_with_relevance_scores(query=query, k=1, kwargs={"score_threshold": 0.8})
# qt_end = timer()

# print("====================================================")
# print("Result: ")
# print(len(result))
# for r in result: 
#     print(r)
#     print(r[0].metadata)
#     print("https://wiki.sql.com.my/wiki/"+quote(r[0].metadata["source"]))
# print("====================================================")
# print("Time: ")
# print("Initialize time(s): ", round(t1_end - t1_start, 2))
# print("Query time(s): %.2f" % round(qt_end - qt_start, 2))



# Define vector store db as retriever
retriever = db.as_retriever(
    search_type="mmr",  # similarity, mmr
    search_kwargs={"k": 3},
)

# Define llm
callback = StreamingStdOutCallbackHandler()
llm = Ollama(model="llama2", base_url="http://host.docker.internal:11434", timeout=3000, temperature=0, callbacks=[callback])

# Define prompt template for llm
template = """You are an assistant for question-answering tasks. 
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question} 

Helpful Answer: """
custom_prompt = PromptTemplate.from_template(template=template)

# RetrievalQA (use RetrievalQAWithSourcesChain to retrieve relevant data)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff", return_source_documents=True, chain_type_kwargs={"prompt": custom_prompt})
# qa_chain = RetrievalQAWithSourcesChain.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff", return_source_documents=True, chain_type_kwargs={"prompt": custom_prompt})

result = qa_chain.invoke({"query": "How to create a debit note?"})
# result = qa_chain({"question": "How to create a debit note?", "context": ""})
# for chunk in result["result"]:
#     print(chunk)

# print(result["source_documents"])
# print(result)
# for d in result["source_documents"]:
#     print(d.metadata)
#     print("https://wiki.sql.com.my/wiki/"+quote(d.metadata["source"]))

# print(result)
# print(result["result"])


Thanks for asking! To create a debit note, you can follow these steps:

1. Go to the "Debit Note" menu in the SQL Accounting system.
2. Click on the "New DN" button to create a new debit note.
3. Enter the customer code and amount to be debited.
4. Select the project type (document or detail) and enter the project name.
5. Attach any supporting documents if necessary.
6. Click "Save" to save the debit note.

That's it! You have successfully created a debit note in SQL Accounting. If you have any questions or need further assistance, feel free to ask.